In [40]:
import csv
from itertools import combinations
import pandas as pd


In [41]:
dataf = pd.read_csv("dataset2.csv")
dataf

,Watcher,videoCategoryId,videoCategoryLabel,definition
0,Billy,29,Nonprofits & Activism,hd
1,Leila,22,People & Blogs,sd
2,Billy,22,People & Blogs,sd
3,Mark,24,Entertainment,hd
4,Billy,24,Entertainment,hd
...,...,...,...,...
110,Jeff,24,Entertainment,hd
111,Billy,26,Howto & Style,sd
112,Clark,28,Science & Technology,hd
113,Billy,24,Entertainment,sd


In [42]:
df= dataf.set_index('Watcher')
df.loc['Billy']

,videoCategoryId,videoCategoryLabel,definition
Watcher,,,
Billy,29,Nonprofits & Activism,hd
Billy,22,People & Blogs,sd
Billy,24,Entertainment,hd
Billy,28,Science & Technology,hd
Billy,26,Howto & Style,sd
Billy,24,Entertainment,sd


In [43]:
df.index.unique()

Index(['Billy', 'Leila', 'Mark', 'Jane', 'Babs', 'Jeff', 'Naomy', 'Flo',
       'Phoebe', 'Rachel', 'Marie', 'Ross', 'Adam', 'Ben', 'Brad', 'Jess',
       'Cyrus', 'Monica', 'Joey', 'Sloane', 'Natoo', 'Ze', 'Liza', 'Ibra',
       'Bob', 'Kat', 'Chandler', 'Alex', 'Jack', 'Penny', 'Squeezy', 'Sheldon',
       'Leonard', 'Amy', 'Howard', 'Radjesh', 'Seb', 'Loan', 'Clark'],
      dtype='object', name='Watcher')

In [44]:
df.videoCategoryLabel.unique()

array(['Nonprofits & Activism', 'People & Blogs', 'Entertainment',
       'News & Politics', 'Science & Technology', 'Education', 'Music',
       'Travel & Events', 'Film & Animation', 'Sports', 'Gaming',
       'Comedy', 'Howto & Style'], dtype=object)

In [45]:
[df.loc['Flo']['videoCategoryLabel']]

['Science & Technology']

In [46]:
def organize_df(dataframe, new_index, target):
        new_df = dataframe.set_index(new_index)
        user_categs = {}
        for each in new_df.index.unique():
                if(isinstance(new_df.loc[each][target], str)): 
                        user_categs[each] = [new_df.loc[each][target]]
                else: user_categs[each] = list(set(new_df.loc[each][target]))
        return user_categs

In [47]:
organize_df(dataf, "Watcher", "videoCategoryLabel")

{'Billy': ['Howto & Style',
  'Science & Technology',
  'People & Blogs',
  'Entertainment',
  'Nonprofits & Activism'],
 'Leila': ['Science & Technology',
  'People & Blogs',
  'Film & Animation',
  'Nonprofits & Activism',
  'Comedy'],
 'Mark': ['Science & Technology',
  'Education',
  'People & Blogs',
  'Entertainment',
  'Nonprofits & Activism'],
 'Jane': ['People & Blogs', 'Entertainment', 'Education'],
 'Babs': ['People & Blogs'],
 'Jeff': ['Science & Technology',
  'Education',
  'Gaming',
  'News & Politics',
  'Entertainment'],
 'Naomy': ['People & Blogs',
  'Music',
  'Science & Technology',
  'Nonprofits & Activism'],
 'Flo': ['Science & Technology'],
 'Phoebe': ['People & Blogs', 'Science & Technology'],
 'Rachel': ['People & Blogs', 'Science & Technology'],
 'Marie': ['People & Blogs', 'Entertainment'],
 'Ross': ['Education', 'News & Politics'],
 'Adam': ['People & Blogs'],
 'Ben': ['People & Blogs'],
 'Brad': ['People & Blogs',
  'Music',
  'Science & Technology',
  'Non

In [48]:
def frequence(items_lst, trans, check=False):
    items_counts = dict()
    for i in items_lst:
        temp_i = {i}
        if check:
            temp_i = set(i)
        for j in trans.items():
            if temp_i.issubset(set(j[1])):
                if i in items_counts:
                    items_counts[i] += 1
                else:
                    items_counts[i] = 1
    return items_counts

In [49]:
def support(items_counts, trans):
    support = dict()
    total_trans = len(trans)
    for i in items_counts:
        support[i] = items_counts[i]/total_trans
    return support

In [50]:
def association_rules(items_grater_then_min_support):
    rules = []
    dict_rules = {}
    for i in items_grater_then_min_support:
        dict_rules = {}
        if type(i) != type(str()):
            i = list(i)
            temp_i = i[:]
            for j in range(len(i)):
                k = temp_i[j]
                del temp_i[j]
                dict_rules[k] = temp_i
                temp_i = i[:]
        rules.append(dict_rules)
    temp = []
    for i in rules:
        for j in i.items():
            if type(j[1]) != type(str()):
                temp.append({tuple(j[1])[0]: j[0]})
            else:
                temp.append({j[1]: j[0]})
    rules.extend(temp)
    return rules

In [51]:
def confidence(associations, d, min_confidence, N):
    ans = {}
    for i in associations:
        for j in i.items():
            if type(j[0]) == type(str()):
                left = {j[0]}
            else:
                left = set(j[0])
            if type(j[1]) == type(str()):
                right = {j[1]}
            else:
                right = set(j[1])
            for k in d:
                if type(k) != type(str()):
                    if left.union(right) - set(k) == set():
                        up = d[k]
                    if len(right) == len(set(k)) and right - set(k) == set():
                            down = d[k]
                else:
                    if len(right) >= len({k}):
                        if right - {k} == set():
                            down = d[k]
                    elif len(right) <= len({k}):
                        if {k} - right == set():
                            down = d[k]
            if up/down >= min_confidence:
                ans[tuple(left)[0]] = right, up/down, up/N, down/N
    return ans   

In [52]:
def correlation_rule(associations, d, min_lift, N):
    ans = {}
    for i in associations:
        for j in i.items():
            if type(j[0]) == type(str()):
                left = {j[0]}
            else:
                left = set(j[0])
            if type(j[1]) == type(str()):
                right = {j[1]}
            else:
                right = set(j[1])
            for k in d:
                if type(k) != type(str()):
                    if left.union(right) - set(k) == set():
                        up = d[k]
                    if len(right) == len(set(k)) and right - set(k) == set():
                            down = d[k]
                else:
                    if len(right) >= len({k}):
                        if right - {k} == set():
                            down = d[k]
                    elif len(right) <= len({k}):
                        if {k} - right == set():
                            down = d[k]
            if (up/down)/(down/N) >= min_lift:
                ans[tuple(left)[0]] = right, up/down, up/N, down/N, (up/down)/(down/N)
    return ans   

In [53]:
def apriori(file_loc,min_support=0.01, min_confidence=0.7, min_lift=1):
    df = pd.read_csv(file_loc)
    trans = organize_df(df, "Watcher", "videoCategoryLabel")
    
    number_of_trans = [len(i) for i in trans.values()]
    items_lst = set()
    
    itemcount_track = list()    
    
    for i in trans.values():
        for j in i:
            items_lst.add(j)
    
    store_item_lst = list(items_lst)[:]
    items_grater_then_min_support = list()
    items_counts = frequence(items_lst, trans)
    itemcount_track.append(items_counts)
    items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})
    
    for i in range(2, max(number_of_trans)+1):
        item_list = combinations(items_lst, i)
        items_counts = frequence(item_list, trans, check=True)
        itemcount_track.append(items_counts)
        if list({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support}.keys()) != []:
            items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})
        
    d = {}
    {d.update(i) for i in itemcount_track}
    associations = association_rules(items_grater_then_min_support[len(items_grater_then_min_support)-1])
    associations_grater_then_confidence = confidence(associations, d, min_confidence, len(trans.keys()))
    correlation_rules = correlation_rule(associations,d, min_lift,len(trans.keys()))
    return associations_grater_then_confidence, correlation_rules

In [54]:
res = apriori('dataset2.csv',0.01, 0.7, 0.2)
#pd.DataFrame.from_dict(res[0], orient='index', columns=['association_rule','confidence', 'support_asso12', 'support_asso1'])
pd.DataFrame.from_dict(res[1], orient='index', columns=['association_rule','confidence', 'support_asso12', 'support_asso1' ,'lift'])

,association_rule,confidence,support_asso12,support_asso1,lift
Science & Technology,{Nonprofits & Activism},0.111111,0.025641,0.230769,0.481481
Sports,"{Science & Technology, Education, People & Blogs, News & Politics, Nonprofits & Activism}",1.000000,0.025641,0.025641,39.000000
Education,"{Science & Technology, Sports, People & Blogs, News & Politics, Nonprofits & Activism}",1.000000,0.025641,0.025641,39.000000
People & Blogs,"{Science & Technology, Sports, Education, News & Politics, Nonprofits & Activism}",1.000000,0.025641,0.025641,39.000000
News & Politics,"{Science & Technology, Sports, Education, People & Blogs, Nonprofits & Activism}",1.000000,0.025641,0.025641,39.000000
Nonprofits & Activism,"{Science & Technology, Sports, Education, People & Blogs, News & Politics}",1.000000,0.025641,0.025641,39.000000


In [55]:
from collections import Counter

# fonction qui convertis le dataset au format adapté pour l'execution de l'algo apriori
def create_data_table(df):
    # On enleve les espaces pour eviter des bugs lors de l'execution de l'algo
    for d in df["videoCategoryLabel"].unique():
        dd = d.replace(" ", "_")
        df["videoCategoryLabel"] = df["videoCategoryLabel"].replace(d, dd)

    # Pour chaque transaction (watcher) on lui associes ses items (videoCategoryLabel)
    data = dict()
    for d in df["Watcher"].unique():
        t = df.loc[df["Watcher"] == d]
        data[d] = list(set(t["videoCategoryLabel"]))

    return data

# fonction qui retourne une liste d'item dans le meme format que dans lequel ils se trouvent dans le dictionaire de base
def item_format(item):
    item_list = list(item.split("'"))
    special_characters = "[', ']"
    # les items etant stocker sours le formet ['X'] on le rend sous le format X
    item_list_format = [i for i in item_list if  i not in special_characters]
    return item_list_format

# fonction qui crée les tables C1, C2, C3,...,Ck
def create_ck_table(data, lk, k):
    c = Counter() # structure pythonique pour conter les objets

    if k == 1: # Dans le cas ou on construit C1 on récupére les items de notre dataset
        item_set = list(set(sum(data.values(), [])))
    else: # sinon on pour k >= 2 on les récupère de la table L(k-1)
        item_set = set()
        temp = list(lk)
        # on réalise des k-itemset unique en faisant des unions avec les objets de la table L(k-1)
        for i in range(0,len(temp)):
            for j in range(i+1,len(temp)):
                t = {z for z in item_format(temp[i])}.union({w for w in item_format(temp[j])})
                if(len(t) == k):
                    t = sorted(t)
                    item_set.add(str(t))
        item_set = list(item_set)

    # on compte le support de chaque k-itemset obtenue
    for i in item_set:
        c[i] = 0
        for d in data.values():
            if all(item in d for item in item_format(i)):
                c[i] += 1
    
    return c

# fonction qui crée les tables L1, L2, L3,...,Lk
def create_lk_table(data, ck, k, s):
    l = Counter()
    # On conserve uniquement les k-itemset de la table Ck qui vérifie le min support
    for i in ck:
        if(ck[i] >= s):
            l[str(i)] += ck[i]
    return l

# fonction qui permet de sauvegarder la table Lk dans l'ensemble L
def save_lk_table(lk, k):
    final = []
    for i in lk:
        i_set = set()
        for it in item_format(i):
            i_set.add(it)
        final.append(i_set)
    return final

# execution de l'algo apriori
def apriori(data,s):
    min_s = len(data) * s # calcule du minimum support
    final = [] # l'ensemble final L
    ck = Counter() # Table Ck
    lk = Counter() # Table Lk

    #On fixe la limite a 1000 pour etre sur de terminer l'execution de l'algorithme
    for k in range(1,1000):
        ck = create_ck_table(data,lk,k)
        if len(ck) == 0: # si la table Ck est vide on termine l'algo
            break

        lk = create_lk_table(data,ck,k,min_s)
        if len(lk) == 0: # si la table Lk est vide on termine l'algo
            break
        
        # On sauvegarde les k-itemset de la table Lk dans l'ensemble L
        l_items = save_lk_table(lk,k)
        for li in l_items:
            final.append(li)
    
    return final

# fonction qui combine tout les items de l'ensemble L entre eux pour obtenir toutes les combinaisons possibles
def pair_up(items):
    pairs = []
    for i in range(len(items)):
        for j in range(len(items)):
            pairs.append((items[i],items[j]))
    return pairs

# fonction qui retournes l'ensembles des régles possibles
# une régle est sous la forme {X --> Y} avec X et Y des itemset
def make_rules(items):
    rules = pair_up(items) # on récupère toutes les combinaisons d'itemset possible
    final_rules = list()

    # on filtres les combinaisons qui sont acceptables comme regles
    for r in rules :
        X = list(r[0])
        Y = list(r[1])
        # Dans le cas ou X intersection Y != {} on retire les items en commun de Y 
        for x in X:
            if x in Y:
                Y.remove(x)
        # Dans le cas ou la régle n'existe pas dèja et que l'itemset Y n'est pas vide après lui avoir
        # retiré les items commun on sauvegarde la régle
        if (X,Y) not in final_rules and len(Y) != 0:
            final_rules.append((X,Y))

    return final_rules

# fonction qui retourne les régles ayant une confiance supperieur a la confiance minimum 
# elle retourne aussi pour chaque regle sa confiance et son lift
def association_correlation_rules(data, items, min_conf):
    table = []
    rules = make_rules(items) # recupere les regles 
    min_c = min_conf * len(data.values()) # on calcule la confiance minimum

    # pour chaque regle on calcule sa confiance et on vérifie si elle est sup a la conf min
    for fr in rules:
        x, y = fr # on recupere les itemsets de la regle par exemple pour la regle {I1, I2} --> {I3, I4}
                # on obtient x = {I1, I2} et y = {I3, I4}

        xy = sum(fr,[]) # transforme la regle de {I1, I2} --> {I3, I4} a {I1, I2, I3, I4}

        count_x, count_y, count_xy = 0, 0, 0 # on initialise un compteur pour chaque itemset

        # on remet les espaces enlever au debut pour l'affichage final
        str_x, str_y = str(set(x)).replace("_", " "), str(set(y)).replace("_", " ")
        rule = str_x +" ---> "+ str_y

        # On calcule la frequence de chaque itemset dans notre dataset
        for d in data.values():
            if x[0] in d:
                count_x += 1
            if y[0] in d:
                count_y += 1
            check =  all(item in d for item in xy)
            if check:
                count_xy += 1
        
        # on calcule leur support 
        support_x = count_x / len(data.values())
        support_y = count_y / len(data.values())
        support_xy = count_xy / len(data.values())

        conf = support_xy / support_x  # On calcule la confiance de la regle 
        lift = support_xy / (support_x * support_y) # On calcule le lift de la regle

        if (conf * len(data.values()) >= min_c): # si la confiance de la regle >= min_c on la sauvegarde avec sa confiance et son lift
            table.append([rule, str(int(conf*100))+"%", "{:.2f}".format(round(support_xy, 2))])
    return table

# Version final de l'algo regroupant toute les fonctions
def algorithme_apriori(data,min_support,min_confidence):
    L = apriori(data,min_support)
    return association_correlation_rules(data, L, min_confidence)

# Retourne les conséquents (Y) de toutes les règles avec un item particulier comme antécédant (X)
def get_recommendation(item, rules):
    recomendations = []
    for r in rules :
        rule = r[0].split(" ---> ")
        X = rule[0]
        Y = rule[1]
        if X == item:
            recomendations.append(Y)
    return recomendations



In [56]:
df = pd.read_csv("dataset2.csv")
data = create_data_table(df)

rules = algorithme_apriori(data, 0.2, 0.2)

pd.set_option('display.max_colwidth', None)

association_rules = pd.DataFrame(rules, columns = ["Rule","Confidence","Lift"])
print(pd.DataFrame(rules, columns = ["Rule","Confidence","Lift"]))

print(get_recommendation("{'Education'}", rules))

                                                                                      Rule  \
0                                        {'People & Blogs'} ---> {'Nonprofits & Activism'}   
1                                         {'People & Blogs'} ---> {'Science & Technology'}   
2                {'People & Blogs'} ---> {'Nonprofits & Activism', 'Science & Technology'}   
3                                        {'Nonprofits & Activism'} ---> {'People & Blogs'}   
4                                             {'Nonprofits & Activism'} ---> {'Education'}   
5                                  {'Nonprofits & Activism'} ---> {'Science & Technology'}   
6                {'Nonprofits & Activism'} ---> {'People & Blogs', 'Science & Technology'}   
7                                                    {'Education'} ---> {'People & Blogs'}   
8                                             {'Education'} ---> {'Nonprofits & Activism'}   
9                                              {'Education'}

In [57]:
columns=['User','Videos Category']
rows=[]
for key in data.keys():
    row=[]
    row.append(key)
    row.append(data[key])
    rows.append(row)
pd.DataFrame(rows,columns=columns)

,User,Videos Category
0,Billy,"[People_&_Blogs, Nonprofits_&_Activism, Howto_&_Style, Entertainment, Science_&_Technology]"
1,Leila,"[People_&_Blogs, Nonprofits_&_Activism, Film_&_Animation, Comedy, Science_&_Technology]"
2,Mark,"[People_&_Blogs, Nonprofits_&_Activism, Education, Entertainment, Science_&_Technology]"
3,Jane,"[Entertainment, People_&_Blogs, Education]"
4,Babs,[People_&_Blogs]
5,Jeff,"[Education, Gaming, Entertainment, Science_&_Technology, News_&_Politics]"
6,Naomy,"[Music, People_&_Blogs, Nonprofits_&_Activism, Science_&_Technology]"
7,Flo,[Science_&_Technology]
8,Phoebe,"[People_&_Blogs, Science_&_Technology]"
9,Rachel,"[People_&_Blogs, Science_&_Technology]"


In [58]:
association_rules

,Rule,Confidence,Lift
0,{'People & Blogs'} ---> {'Nonprofits & Activism'},25%,0.21
1,{'People & Blogs'} ---> {'Science & Technology'},54%,0.44
2,"{'People & Blogs'} ---> {'Nonprofits & Activism', 'Science & Technology'}",25%,0.21
3,{'Nonprofits & Activism'} ---> {'People & Blogs'},88%,0.21
4,{'Nonprofits & Activism'} ---> {'Education'},33%,0.08
5,{'Nonprofits & Activism'} ---> {'Science & Technology'},100%,0.23
6,"{'Nonprofits & Activism'} ---> {'People & Blogs', 'Science & Technology'}",88%,0.21
7,{'Education'} ---> {'People & Blogs'},75%,0.15
8,{'Education'} ---> {'Nonprofits & Activism'},37%,0.08
9,{'Education'} ---> {'Science & Technology'},62%,0.13
